In [1]:
import os
import torch
import numpy as np
from matplotlib import pyplot as plt
from neulight.dataset.free_viewpoint import FreeViewpointDataset
from torch.utils.data import DataLoader

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
dtype = torch.float32
device = torch.cuda.current_device()
batch_size = 4
num_workers = 2
free_viewpoint_scene_path = "../data/neulight/Salon2"

In [3]:
free_viewpoint_train_dataset = FreeViewpointDataset(
    scene_path=free_viewpoint_scene_path,
    dataset_type="train",
)
free_viewpoint_train_dataloader = DataLoader(
    free_viewpoint_train_dataset,
    batch_size=batch_size,
    shuffle=False,
    collate_fn=free_viewpoint_train_dataset.collate_fn,
    num_workers=num_workers,
    persistent_workers=True,
)

In [4]:
free_viewpoint_test_dataset = FreeViewpointDataset(
    scene_path=free_viewpoint_scene_path,
    dataset_type="test",
)
free_viewpoint_test_dataloader = DataLoader(
    free_viewpoint_test_dataset,
    batch_size=batch_size,
    shuffle=False,
    collate_fn=free_viewpoint_test_dataset.collate_fn,
    num_workers=num_workers,
    persistent_workers=True,
)

In [5]:
train_batch = free_viewpoint_train_dataset.move_batch_to(
    next(iter(free_viewpoint_train_dataloader)),
    device=device,
    dtype=dtype,
)

In [12]:
train_batch["labels"]["colors"]

tensor([[[0.4039, 0.2824, 0.1686],
         [0.6549, 0.5137, 0.3804],
         [0.6902, 0.5451, 0.4314],
         ...,
         [0.9490, 0.7333, 0.4784],
         [0.1725, 0.1451, 0.1059],
         [0.5647, 0.3765, 0.2196]],

        [[0.4039, 0.3059, 0.1843],
         [0.6392, 0.4196, 0.2275],
         [0.4667, 0.3569, 0.2000],
         ...,
         [0.5412, 0.4157, 0.2314],
         [0.3804, 0.3255, 0.2196],
         [0.2980, 0.2275, 0.1804]],

        [[0.2627, 0.1608, 0.0941],
         [0.9922, 0.9490, 0.6000],
         [0.5059, 0.3922, 0.2353],
         ...,
         [0.6118, 0.4118, 0.2275],
         [1.0000, 1.0000, 0.9647],
         [0.6275, 0.4000, 0.2118]],

        [[0.5490, 0.4196, 0.2510],
         [0.1608, 0.1137, 0.0588],
         [0.5922, 0.4039, 0.2235],
         ...,
         [0.7216, 0.5725, 0.3961],
         [0.2902, 0.2235, 0.1529],
         [0.2471, 0.1765, 0.1216]]], device='cuda:0')

In [6]:
test_batch = free_viewpoint_test_dataset.move_batch_to(
    next(iter(free_viewpoint_test_dataloader)),
    device=device,
    dtype=dtype,
)

In [7]:
vis = free_viewpoint_test_dataset.visualize_batch(test_batch)